In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from filters import high_pass_filter, denoise_signal, sinewave_shift, drop_data, count_peaks
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import gc

In [3]:
train_meta = pd.read_csv('data/metadata_train.csv')

In [4]:
train_meta['peaks'] = 0
filtered_dataset = None

train_chunks = [(0, 999), (1000, 1999), (2000, 2999), (3000, 3999),(4000, 4999),
                (5000, 5999), (6000, 6999), (7000, 7999), (8000, 8711)]

for chunk_id, chunk in enumerate(train_chunks):
    data_subset = pq.read_pandas('data/train.parquet', 
                                 columns=[str(x) for x in range(chunk[0], chunk[1]+1)]).to_pandas()
    
    for idx, column in enumerate(data_subset.columns):
        new_signal, cutpoint = sinewave_shift(data_subset[column])
        hp_filtered_signal = high_pass_filter(new_signal)
        denoised_sig = denoise_signal(hp_filtered_signal)
        cut_signal = drop_data(denoised_sig)

        train_meta['peaks'].loc[int(column)] = count_peaks(cut_signal)
    
        if isinstance(filtered_dataset, type(None)):
            filtered_dataset = pd.DataFrame(data=cut_signal, columns=[column])
        else:
            filtered_dataset[column] = cut_signal
        
        print("Finished filtering signal Id {}".format(column), end="\r")

    
    filtered_train = pa.Table.from_pandas(filtered_dataset)
    pq.write_table(filtered_train, 'data/filtered_train_{}.parquet'.format(chunk_id))
        
    filtered_dataset = None

In [5]:
train_meta.to_csv('data/filtered_metadata_train.csv', index=False)